# Dependancies

In [ ]:
import pandas as pd
import numpy as np

import datetime as dt

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors

import math
from scipy.stats import skew
from scipy.stats import stats
from scipy import fftpack

try:
    import tkinter.ttk
    from tkinter import *
except:
    from tkinter import *
    
from tkinter import messagebox
from tkinter import filedialog

from pandas.plotting import scatter_matrix
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from pandas.plotting import bootstrap_plot

import os
import sys

from IPython.core.display import HTML

# Classes

In [ ]:
class TestData():
    
    data_columns_list=list()
    
    def __init__(self,datafile):
        self.datafile=datafile
        
    def get_data(self):
        self.data=pd.read_csv(self.datafile, skiprows=None)
        pretty_display(self.data)
        TestData.data_columns_list=self.data.columns.tolist()
        print(f'THE HEADERS:{TestData.data_columns_list}')
        self.drop_unused_columns()
        
        
    def drop_unused_columns(self):
        # All of the columns named UNUSED
        p=self.data.loc[:, self.data.columns.str.contains('Unnamed')].head()
        self.data.drop(p,axis=1,inplace=True)
        pretty_display(self.data)
        self.clean_data()
        
    def clean_data(self):
        #replacing header spaces with underscores 
        self.data.columns=self.data.columns.str.replace(' ','_')

        #replacing header dashes with underscores
        self.data.columns=self.data.columns.str.replace('-','_')
        self.data.columns=self.data.columns.str.replace(':','')
        self.data.columns=self.data.columns.str.replace('@','')
        pretty_display(self.data)
        
        
    def plot_data(self):
    
        pd.set_option('precision',1)
        plt.style.use('ggplot')

        fig, ax = plt.subplots(2,1,figsize=(15,10))

        plt.suptitle('The Graph')

        the_labels=self.data.columns.tolist()
#         print(the_labels)
        
        xlow=self.data['Time_in_Seconds'].min()
        xhigh=self.data['Time_in_Seconds'].max()
        pressure_max=round(self.data['Port_3'].max(),ndigits=1)

        ax[0]=plt.subplot(211)
        ax[0].plot(self.data['Time_in_Seconds'],self.data['Port_3'])
        ax[0].axhline(y=pressure_max,linewidth=4, color='black',linestyle="--")
        ax[0].set_xlabel('Time in Sesconds')
        ax[0].set_ylabel('Pressure (PSIG)')
        ax[0].set_xlim(xlow,xhigh)
        ax[0].set_title(f'Max Pressure:{pressure_max} PSIG',loc='left')
        ax[0].legend()
        
        ax[1]=plt.subplot(212)
        ax[1].plot(self.data['Time_in_Seconds'],self.data['Port_3'])
        ax[1].set_xlabel('Time in Sesconds')
        ax[1].set_ylabel('Pressure (PSIG)')
        ax[1].set_xlim(xlow,xhigh)
        ax[1].set_title('Pressure',loc='left')
        ax[1].legend()

        plt.show()
        

In [ ]:
class UserInterface():
    """Parent class for the UI. Instantiates the composit Window"""
    def __init__(self):
        UI(None)
        mainloop()


class UI(Tk):
    """User Interface."""
    
    now=dt.date.today().strftime('%B %d, %Y')
    time_of_day=dt.datetime.today().strftime('%I:%M:%S %p')
    
        
    initialdir=os.getcwd()
    
    def __init__(self,parent,*args,**kargs):
        """Create the UI Window"""
        Tk.__init__(self,parent,*args,**kargs)
        
        self.parent=parent
        self.initialize()
        self.banner=Label(self,text=f'DRILLING CONTROLS DATA PLOTS & ANALYSIS - {UI.now}',fg='white',bg='blue',font='Ariel 30 bold')
        self.banner.grid(row=0,column=0, columnspan=2)

    def initialize(self):
            """Set-up and configure the UI window"""
            self.title('UML Project')
            the_window_width=self.winfo_screenwidth()
            the_window_height=self.winfo_screenheight()
    #         self.configure(width=the_window_width,height=the_window_height)
    #         the_window_width=1200
    #         the_window_height=700
            self.geometry(f'{the_window_width}x{the_window_height}+0+0')
    #         self.attributes('-fullscreen', True)
            self['borderwidth']=4
            self['bg']='blue'
            self.menubar=Menu(self)
            self.menubar.add_command(label="Exit",font='ariel',command=self.bye_bye)
            self.config(menu=self.menubar)
            self.makeFields()
    
    def bye_bye(self):
        """Close the UI Window on menu Exit"""
        self.destroy()
        
    def makeFields(self):
        """Generate the fields and buttons"""
        # make the frame
        self.frame1=Frame(self.parent)
        self.frame1['background']='green'
        self.frame1['relief']='raised'
        self.frame1['borderwidth']=10
        self.frame1.grid(row=1,column=1)
        banner_text='Drilling Controls-Data Plotting & Analysis'
        frame_banner=Label(self.frame1,text=banner_text,fg='white',bg='green',font='Ariel 15 bold')
        frame_banner.grid(row=0,column=0,columnspan=5,pady=15)
        self.v1=StringVar()
        self.txt_file=Entry(self.frame1,bg='yellow',font='Ariel 15 bold',width=30,textvariable=self.v1)
        self.txt_file.grid(row=3,column=0,sticky=E)
        
        # Buttons
        
        self.access_dir=Button(self.frame1, text="Open Directory",bg='blue',fg='yellow',relief='raised',state='normal',command=self.open_directory)
        self.access_dir.grid(row=5,column=0,sticky=W,pady=15)
        
        # Header list box
        self.header_list=Listbox(self.frame1)
        self.header_list['selectmode']=EXTENDED
        self.header_list.grid(row=6,column=0,sticky=W,pady=15)
        
        
        
        
    def open_directory(self):
        """Opens the directory folder for user to access"""
        
        x=filedialog.askopenfilename(initialdir = UI.initialdir,title = "Directory",
                                     filetypes = (("csv files","*.csv"),("all files","*.*")))
        
        print(x)
        
        self.set_directory(x)

        the_file_name=x.split(sep='/')[-1].split('.')[0]
        self.v1.set(the_file_name)
        dataFile=f'{the_file_name}.csv'
        print(dataFile)
        
        
        
        x=TestData(dataFile)
        x.get_data()
        
        self.header_list.delete(0,END)
        header_list=TestData.data_columns_list
        print(f'HEADERS:{header_list}')
        for i in header_list:
            self.header_list.insert(END,i)
        
        x.plot_data()
    
        self.header_list.bind("<<ListboxSelect>>",lambda x:self.selected_columns())
            
    def selected_columns(self):
        selection=self.header_list.curselection()
        c=self.header_list.get(ANCHOR)
        print(c)
    
        
    def set_directory(self,selectedFile):
        """Changes the initialdir for the filedialog window and uses the same directory for files saved."""
        
        filename=selectedFile.split(sep='/')[-1]
        
        the_dir=selectedFile.split(sep=filename)
        
        UI.initialdir=the_dir[0]

# Functions

In [ ]:
def pretty_display(dataframe):
    display(HTML(dataframe.head().to_html()))

# Main

In [ ]:
if __name__ == '__main__':
    UserInterface()

In [ ]:
x=TestData('999-20_test7_100hz.csv')

In [ ]:
x.get_data()

In [ ]:
x.plot_data()

In [8]:
x=[1,2,3,4]

In [9]:
x

[1, 2, 3, 4]

In [10]:
x.clear()

In [11]:
x

[]

In [12]:
x=[]

In [13]:
x.append(1)

In [14]:
x

[1]

In [15]:
x.append(2)

In [16]:
x

[1, 2]

In [17]:
x.pop()

2

In [19]:
x

[1]

In [20]:
x[0]

1